In [4]:
import import_ipynb
import IntimeLibrary as il
import FlieLibrary as fl
import pandas as pd
import numpy as np

In [5]:
print(fl.flight_from_cju.head(10))
print(fl.flight_to_gmp.head(10))
print(fl.flight_from_gmp.head(10))
print(fl.flight_to_cju.head(10))

         날짜     항공사    계획    출발  현황  time          TM  delayTime weekend
0  20181001  아시아나항공  6:30  6:37  출발     6  2018100106        7.0     Mon
1  20181001  아시아나항공  6:40  6:44  출발     6  2018100106        4.0     Mon
2  20181001    대한항공  7:00  7:09  출발     7  2018100107        9.0     Mon
3  20181001     진에어  7:05  7:21  출발     7  2018100107       16.0     Mon
4  20181001    제주항공  7:40  7:54  출발     7  2018100107       14.0     Mon
5  20181001  아시아나항공  7:55  8:02  출발     7  2018100107        7.0     Mon
6  20181001   티웨이항공  7:55  8:05  출발     7  2018100107       10.0     Mon
7  20181001    대한항공  8:00  8:15  출발     8  2018100108       15.0     Mon
8  20181001     진에어  8:05  8:19  출발     8  2018100108       14.0     Mon
9  20181001   이스타항공  8:10  8:40  지연     8  2018100108       30.0     Mon
         날짜     항공사    계획    도착  현황  time          TM  delayTime weekend
0  20181001  아시아나항공  7:40  7:25  도착     7  2018100107      -15.0     Mon
1  20181001  아시아나항공  7:50  7:29  도착     7  20181001

In [6]:
print(fl.flight_from_cju['delayTime'].isnull().sum())
print(fl.flight_to_gmp['delayTime'].isnull().sum())
print(fl.flight_from_gmp['delayTime'].isnull().sum())
print(fl.flight_to_cju['delayTime'].isnull().sum())

309
326
288
313


In [7]:
temp1 = fl.flight_from_cju.dropna(axis=0)
temp2 = fl.flight_to_gmp.dropna(axis=0)
temp3 = fl.flight_from_gmp.dropna(axis=0)
temp4 = fl.flight_to_cju.dropna(axis=0)

In [8]:
print(temp1['delayTime'].isnull().sum())
print(temp2['delayTime'].isnull().sum())
print(temp3['delayTime'].isnull().sum())
print(temp4['delayTime'].isnull().sum())

0
0
0
0


In [9]:
print(temp1['delayTime'].max(),"/",temp1['delayTime'].min())
print(temp2['delayTime'].max(),"/",temp2['delayTime'].min())
print(temp3['delayTime'].max(),"/",temp3['delayTime'].min())
print(temp4['delayTime'].max(),"/",temp4['delayTime'].min())

387.0 / -165.0
368.0 / -166.0
370.0 / -141.0
359.0 / -550.0


In [10]:
print(temp4.sort_values(by=['delayTime'],axis=0,ascending=True).head(20))

             날짜     항공사     계획     도착  현황  time          TM  delayTime weekend
42713  20190916  아시아나항공  19:55  10:45  도착    19  2019091619     -550.0     Mon
6828   20181125    대한항공  10:10   1:02  지연    10  2018112510     -548.0     SUN
16620  20190216   이스타항공  21:30  12:24  도착    21  2019021621     -546.0     SAT
11860  20190106    에어필립  18:40  16:08  도착    18  2019010618     -152.0     SUN
31483  20190620     진에어  11:35   9:28  도착    11  2019062011     -127.0     THU
11623  20190104    에어필립  19:05  17:20  도착    19  2019010419     -105.0     FRI
12687  20190113    에어필립  18:40  17:03  도착    18  2019011318      -97.0     SUN
30263  20190610    제주항공  16:40  15:30  도착    16  2019061016      -70.0     Mon
17467  20190224    대한항공  13:40  12:43  도착    13  2019022413      -57.0     SUN
7653   20181202    대한항공  10:30   9:35  도착    10  2018120210      -55.0     SUN
349    20181003   이스타항공  19:20  18:45  도착    19  2018100319      -35.0     WED
31239  20190618    제주항공  12:30  11:55  도착    12  201

In [11]:
il.binning(fl.flight_from_cju)
il.binning(fl.flight_to_gmp)
il.binning(fl.flight_from_gmp)
il.binning(fl.flight_to_cju)

In [12]:
from_cju = il.one_hot(fl.flight_from_cju,['weekend'])
to_gmp = il.one_hot(fl.flight_to_gmp,['weekend'])
from_gmp = il.one_hot(fl.flight_from_gmp,['weekend'])
to_cju = il.one_hot(fl.flight_to_cju,['weekend'])

In [14]:
from_cju.head(-10)

,날짜,항공사,계획,출발,현황,time,TM,delayTime,DelayRate,weekend_FRI,weekend_Mon,weekend_SAT,weekend_SUN,weekend_THU,weekend_TUE,weekend_WED
0,20181001,아시아나항공,6:30,6:37,출발,6,2018100106,7.0,1,0,1,0,0,0,0,0
1,20181001,아시아나항공,6:40,6:44,출발,6,2018100106,4.0,1,0,1,0,0,0,0,0
2,20181001,대한항공,7:00,7:09,출발,7,2018100107,9.0,1,0,1,0,0,0,0,0
3,20181001,진에어,7:05,7:21,출발,7,2018100107,16.0,1,0,1,0,0,0,0,0
4,20181001,제주항공,7:40,7:54,출발,7,2018100107,14.0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44082,20191001,티웨이항공,20:30,:,취소,20,2019100120,NaN,NaN,0,0,0,0,0,1,0
44083,20191001,진에어,20:35,20:52,출발,20,2019100120,17.0,1,0,0,0,0,0,1,0
44084,20191001,아시아나항공,20:50,21:54,지연,20,2019100120,64.0,3,0,0,0,0,0,1,0
44085,20191001,제주항공,20:50,21:10,출발,20,2019100120,20.0,1,0,0,0,0,0,1,0
